<a href="https://colab.research.google.com/github/zainiba/The-Collective-Mind/blob/main/150epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
# Read, then decode for py2 compat.
text = open('input1.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
vocab = sorted(set(text))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

I
t
 
s
h


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(optimizer='adam', loss=loss)

(64, 100, 98) # (batch_size, sequence_length, vocab_size)
Prediction shape:  (64, 100, 98)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.586042


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 150

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/150
40/40 [==============================] - 196s 5s/step - loss: 3.5983
Epoch 2/150
40/40 [==============================] - 196s 5s/step - loss: 2.6119
Epoch 3/150
40/40 [==============================] - 195s 5s/step - loss: 2.4046
Epoch 4/150
40/40 [==============================] - 194s 5s/step - loss: 2.2935
Epoch 5/150
40/40 [==============================] - 195s 5s/step - loss: 2.1837
Epoch 6/150
40/40 [==============================] - 193s 5s/step - loss: 2.0735
Epoch 7/150
40/40 [==============================] - 199s 5s/step - loss: 1.9642
Epoch 8/150
40/40 [==============================] - 193s 5s/step - loss: 1.8575
Epoch 9/150
40/40 [==============================] - 193s 5s/step - loss: 1.7597
Epoch 10/150
40/40 [==============================] - 193s 5s/step - loss: 1.6697
Epoch 11/150
40/40 [==============================] - 192s 5s/step - loss: 1.5841
Epoch 12/150
40/40 [==============================] - 193s 5s/step - loss: 1.5141
Epoch 13/150
40/40 [=====

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([64, None]))

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    # model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Art "))

Art that could be rother. On the progress pathern suggests that the machine is in itself is formed in Synthese resulting forms might constitute the "event." In addition, Maciunas's description suggests that the significance of the bring nd in hand.

In closing, there is no telling where my style will time benntes. New labion must have an HSBroadband server, which would need to be different than the low speed, and how long would not be devotely 9  essence in a time thers a series of discrete instructions that were maripulated at the whim of the netr on alization, whereby the more them defle of the planet. (T) in the separation of aitiating system. This project was in close proximity to a few leaf environment DCN. These material soll that talking examples of botang oberal.

Ancient running on the internet where open the audion de creation of massive man made hard with a Marxist interesting way to create a framework for our thoughts, a common scaffolding upon which we can short pieces are